Copyright (c) 2026 [Vital de Nodrest]

The code (Python, shell and PowerShell cells) is under MIT license. Feel free to share and experiment! See LICENSE-CODE.txt in the project root for more information.

Due to their time-consuming and didactic nature, text & image contents are under CC BY-NC-SA 4.0 license. Removal of the author's name or redistribution without credit is prohibited. See LICENSE-DOCS.txt in the project root for more information.

# Introduction to PINNs for Helmholtz problems

This notebook aims at being a basic introduction to PINNs using the example of Helmholtz wave propagation problems.

## Configuration

You can use any recent Python configuration to run this notebook.

The packages are specified in the [requirements](../requirements.txt) file.

In [ ]:
# pip install -r requirements.txt

In [ ]:
# pip install -r requirements.txt

The next subsections provide different configuration scripts depending on your device.

You can uncomment and run the ones you need.


The models in this notebook are:
- small (<< 100k parameters)
- linear (no attention, convolution, *etc*)
- trained on small batch sizes (< 32 most of the time)

*Ergo*, computations should run way faster on CPU than on other hardware. The configuration will be way simpler, as PyTorch runs on CPU by default.

### Linux, CPU

Configuration for Linux devices.

In [ ]:
#pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

### Mac, CPU

Configuration for Mac devices.

In [ ]:
pip install torch torchvision

### Windows, CPU

Configuration for Windows devices.

In [ ]:
#pip install torch torchvision

### Something went wrong?

If your situation doesn't fit any of the subsections, see the [PyTorch installation tutorial](https://pytorch.org/get-started/locally/).

## About PINNs

### Universal approximation theorem

TODO

### Automatic differentiation and physical losses

TODO automatic differentiation & backpropagation



PINNs are based on the following idea: what if we ran the backpropagation a step further, thus reaching the input layer?

In the context of a neural network with physical coordinates as inputs and field variables as outputs, this allows us to compute physically relevant partial derivatives of the model.

They are automatically differentiable themselves, allowing us to run the backpropagation again, yielding higher order derivatives.

These partial derivatives can then be used to compute the residuals of a PDE problem. These residuals themselves are automatically differentiable, enabling adjustment of the model weights *via* backpropagation again.

## A 1D Helmholtz problem

A 1D example problem means fast computations on CPU and highly visual results.

Let's consider the problem:
$$
\frac{d^2u}{dx^2} + k^2 u = 0 ~~ \text{in} ~ ]0,1[
$$

Where $k$ is a real-valued wavenumber. Conveniently, let's suppose that it is not $0$ or a multiple of $2\pi$.

Thanks to usual results about second-order linear differential equations with constant coefficients, we know that solutions to this problem take the following form, with $c_1$ and $c_2$ two constant coefficients:
$$
u(x) = c_1 \cos{kx} + c_2 \sin{kx}
$$

Let's use $\frac{du}{dx}(0) = 0$ and $\frac{du}{dx}(1) = - k \sin{k}$ as boundary conditions. They don't necessarily represent an interesting physical phenomenon, but at least they ensure $c_1 = 1$ and $c_2 = 0$.

Consequently, our problem is well-posed and its only solution is $u \in [0,1] \rightarrow \mathbb{R}$ such that:
$$
\forall x \in [0,1], ~ u(x) = \cos{kx}
$$

This exact solution would not be available in general. It will be used for test purposes, but it will be ignored during training and validation.

## Implementation

The following chapter goes over the PyTorch implementation of a PINN solver using the Helmholtz example.

Feel free to play with the parameters.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### Helmholtz PDE problem

To implement a PDE problem to be solved using a PINN, one needs to write the expressions of its residuals. The model will be trained to minimize these.

These residuals based on partial derivatives are computed using automatic differentation (PyTorch autograd implementation).

Setting the problem wavenumber.

In [ ]:
k = 5

Notation reminder: $f_{\theta}$ is the model with parameters $\theta$.

Residual for a point $x_i$ inside the domain $]0, 1[$:

$$
\frac{\partial^2 f_{\theta}}{\partial x^2}(x_i) + k^2 f_{\theta}(x_i)
$$

In [ ]:
def residual_pde(X: torch.Tensor, U: torch.Tensor) -> torch.Tensor:
    """Batch computing the model PDE residual in the domain ]0,1[.

    Args:
        X (torch.Tensor): n model inputs.
        U (torch.Tensor): n model outputs after respective inferences on X.

    Returns:
        torch.Tensor: n model PDE residuals.
    """

    du_dx = torch.autograd.grad(outputs=U,
                                inputs=X,
                                grad_outputs=torch.ones_like(U), # Shape information for batches
                                create_graph=True, # creating a graph for higher order derivatives
                                retain_graph=True,
                                )[0]
    du_dxx = torch.autograd.grad(outputs=du_dx,
                                 inputs=X,
                                 grad_outputs=torch.ones_like(du_dx), # Shape information for batches
                                 create_graph=True,
    )[0]
    return du_dxx + k**2 * U

Implementing the residual for a point $x_i$ on the left boundary ($i.e.$ $0$ in 1D):

$$
\frac{\partial f_{\theta}}{\partial x}(0)
$$

In [ ]:
def residual_0(X, U):
    du_dx = torch.autograd.grad(outputs=U,
                                inputs=X,
                                grad_outputs=torch.ones_like(U), # Shape information for batches
                                create_graph=True,
                                retain_graph=True,
                                )[0]
    return du_dx

Implementing the residual for a point $x_i$ on the right boundary ($i.e.$ $1$ in 1D):

$$
\frac{\partial f_{\theta}}{\partial x}(1) + k \sin{k}
$$

In [ ]:
def residual_1(X, U):
    du_dx = torch.autograd.grad(outputs=U,
                                inputs=X,
                                grad_outputs=torch.ones_like(U), # Shape information for batches
                                create_graph=True,
                                retain_graph=True,
                                )[0]
    return du_dx + k * torch.sin(k * torch.ones_like(X))

In [ ]:
def u(x):
    return torch.cos(k*x)

### Data

In [ ]:
### Training
X_train_pde = torch.linspace(0, 1, 10 + 2, dtype=torch.float32)[1: -1].unsqueeze_(1)
X_train_0 = torch.tensor([0], dtype=torch.float32).unsqueeze_(1) # left boundary
X_train_1 = torch.tensor([1], dtype=torch.float32).unsqueeze_(1) # right boundary
### Validation
X_validation_pde = torch.linspace(0.01, 0.99, 100, dtype=torch.float32).unsqueeze_(1)
X_validation_0 = torch.tensor([0], dtype=torch.float32).unsqueeze_(1)
X_validation_1 = torch.tensor([1], dtype=torch.float32).unsqueeze_(1)
### Test
X_test = torch.linspace(0, 1, 1000, dtype=torch.float32).unsqueeze_(1)
U_test_exact = u(X_test)
U_test_exact_norm = torch.linalg.vector_norm(U_test_exact)

### Neural network

Let's initialize our neural network.

A typical PINN takes the physical coordinates of the problem as an input and outputs the solution.
In this example, the input is **x** (**1D** space variable) and the output is **u** (**1D** scalar output).

There are many architectural possibilities for the neural network. The simplest choice is a uniform fully-connected neural network with tanh activation functions. By default, each connexion has a weight and a bias.

We cannot use the tanh activation function after the last layer as we need a solution that can reach $1$ and $-1$.

TODO plot

In [ ]:
class FCNN(nn.Module):
    """Simplest fully connected neural network for our problem.
    """
    def __init__(self, input_dim: int=1, output_dim: int=1, width: int=20, hidden_layers: int=3):
        """Initalize the simplest fully-connected neural network.
        All hidden layers have the same width.
        All edges have a weight and a bias.
        Tanh activation functions (except for the last layer).

        Args:
            input_dim (int, optional): Input dimension. Defaults to 1.
            output_dim (int, optional): Output dimension. Defaults to 1.
            width (int, optional): Width of the single layers. Defaults to 20.
            hidden_layers (int, optional): Number of hidden layers. Defaults to 3.
        """
        super().__init__()
        #self.flatten = nn.Flatten()
        layers = [nn.Linear(input_dim, width), nn.Tanh()] # input -> hidden layer 1
        # hidden layer i -> hidden layer i+1
        for _ in range(hidden_layers-1):
            layers.append(nn.Linear(width, width))
            layers.append(nn.Tanh())
        layers.append(nn.Linear(width, output_dim)) # last hidden layer -> output
        self.stack = nn.Sequential(*layers)
    
    def forward(self, x):
        logits = self.stack(x)
        return logits

Feel free to change the parameters, or even write your own class. As an example, how would you write a fully-connected neural network with hidden layers if varying sizes ?

In [ ]:
model = FCNN(width=50)

### Optimization

Let's optimize our model parameters $\theta$ (weight & biases) iteratively using the Adam algorithm with a learning rate of $.001$ to minimize the mean-square-error loss function:

$$L_{\text{MSE}} \left(\theta, (x_i)_{i=1}^N, (y_i)_{i=1}^N \right) = \frac{1}{N} \sum_{i=1}^{N} \left( f_{\theta}(x_i) - y_i \right)^2 $$

Where $f_{\theta}$ is the model parametrized by the vector $\theta$, and we consider $N$ training samples:
- Training points $(x_i)_{i=1}^N$
- Respective solutions $(y_i)_{i=1}^N$

In [ ]:
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss
loss = torch.nn.MSELoss() # the default reduction is "mean", dividing the loss by the number of samples

### Training loop

In [ ]:
# LOOP

num_epochs = 5000 # !!!
train_losses = torch.zeros((3, num_epochs))
validation_losses = torch.zeros((4, num_epochs))
test_metrics = torch.zeros((num_epochs))

best_model_state = None
best_validation_loss = torch.inf

for epoch in range(num_epochs):
    
    ## Training phase
    
    ### Configuration
    model.train()
    optimizer.zero_grad() # reset previously accumulated gradients
    
    ### Optional reampling
    #X_train_pde = torch.rand(size=(10,1)).unsqueeze_(1)
    
    ### Enable automatic differentiation for training and residuals audtodiff
    X_train_pde.requires_grad_(True)
    X_train_0.requires_grad_(True)
    X_train_1.requires_grad_(True)
    
    ### Evaluate the model
    U_train_pde = model(X_train_pde)
    U_train_0 = model(X_train_0)
    U_train_1 = model(X_train_1)
    
    ### Compute residuals
    RES_train_pde = residual_pde(X_train_pde, U_train_pde)
    RES_train_0 = residual_0(X_train_0, U_train_0)
    RES_train_1 = residual_1(X_train_1, U_train_1)
    
    ### Evaluate losses and accumulate the parameter derivatives in place
    train_loss_pde: torch.Tensor = loss(RES_train_pde, torch.zeros_like(RES_train_pde))
    train_loss_pde.backward()
    train_losses[0, epoch] = train_loss_pde.tolist()
    
    train_loss_0 = loss(RES_train_0, torch.zeros_like(RES_train_0))
    train_loss_0.backward()
    train_losses[1, epoch] = train_loss_0.tolist()
    
    train_loss_1 = loss(RES_train_1, torch.zeros_like(RES_train_1))
    train_loss_1.backward()
    train_losses[2, epoch] = train_loss_1.tolist()
    
    ### Perform optimizer step
    optimizer.step()
        
        
    ## Validation phase
    
    ### Configuration
    model.eval()
    
    """
    torch.no_grad() cannot be used because computing 
    """
    
    ### Enable automatic differentiation for residuals audtodiff
    X_validation_pde.requires_grad_(True)
    X_validation_0.requires_grad_(True)
    X_validation_1.requires_grad_(True)
        
    ### Evaluate the model
    U_validation_pde = model(X_validation_pde)
    U_validation_0 = model(X_validation_0)
    U_validation_1 = model(X_validation_1)
    
    ### Compute residuals
    RES_validation_pde = residual_pde(X_validation_pde, U_validation_pde)
    RES_validation_0 = residual_0(X_validation_0, U_validation_0)
    RES_validation_1 = residual_1(X_validation_1, U_validation_1)
    
    ### Evaluate loss
    validation_loss_total = 0
    
    validation_loss_pde: torch.Tensor = loss(RES_validation_pde, torch.zeros_like(RES_validation_pde))
    validation_losses[0, epoch] = validation_loss_pde.tolist()
    validation_loss_total += validation_loss_pde.tolist()
    
    validation_loss_0: torch.Tensor = loss(RES_validation_0, torch.zeros_like(RES_validation_0))
    validation_losses[1, epoch] = validation_loss_0.tolist()
    validation_loss_total += validation_loss_0.tolist()
    
    validation_loss_1: torch.Tensor = loss(RES_validation_1, torch.zeros_like(RES_validation_1))
    validation_losses[2, epoch] = validation_loss_1.tolist()
    validation_loss_total += validation_loss_1.tolist()
    
    
    validation_losses[3, epoch] = validation_loss_total
        
    ### Save the best model
    if validation_loss_total < best_validation_loss:
        best_validation_loss = validation_loss_total
        best_model_state = model.state_dict().copy()
        

    ## Test phase (optional in the loop)
    
    with torch.no_grad():
        U_test = model(X_test)
        test_metrics[epoch] = (torch.linalg.vector_norm(U_test - U_test_exact) / U_test_exact_norm)

### Displaying results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Training history
fig1, ax1 = plt.subplots()
ax1.set_yscale("log")
ax1.plot(train_losses[0, :], label=']0,1[ residual')
ax1.plot(train_losses[1, :], label='0 residual')
ax1.plot(train_losses[2, :], label ='1 residual')
ax1.plot(torch.sum(train_losses, dim=0), label='total residual')
ax1.set_xlabel('epoch')
ax1.set_title('Training losses')
ax1.legend()


In [ ]:
fig2, ax2 = plt.subplots()
ax2.set_yscale("log")
ax2.plot(validation_losses[0, :], label=']0,1[ residual')
ax2.plot(validation_losses[1, :], label='0 residual')
ax2.plot(validation_losses[2, :], label='1 residual')
ax2.plot(validation_losses[3, :], label='total residual')
ax2.legend()
ax2.set_title('Validation losses')


In [ ]:

fig3, ax3 = plt.subplots()
ax3.set_yscale("log")
ax3.plot(test_metrics)


In [ ]:

## Last model
fig4, ax4 = plt.subplots()
ax4.plot(X_test, U_test_exact)
ax4.plot(X_test, U_test)

u_min = torch.min(torch.min(U_test_exact), torch.min(U_test))
u_max = torch.max(torch.max(U_test_exact), torch.max(U_test))
for x in X_train_pde.detach().tolist() + X_train_0.detach().tolist() + X_train_1.detach().tolist():
    ax4.plot([x]*2, [u_min, u_max], color='grey', linewidth=.3)

last_str = f"Last model test metric: {test_metrics[-1].tolist()}"
print(last_str)


In [ ]:

## Best model
with torch.no_grad():
    model.load_state_dict(best_model_state)
    U_test = model(X_test)
    best_test_metric = (torch.linalg.vector_norm(U_test - U_test_exact) / U_test_exact_norm)
fig5, ax5 = plt.subplots()
ax5.plot(X_test, U_test_exact)
ax5.plot(X_test, U_test)
best_str = f"Best model test metric: {best_test_metric.tolist()}"
print(best_str)

## Running

## Further research...

Stochastic PDEs

Multi-physics

Implementations

Gradient pathologies

Loss functions with multiple components

Hyperparameter optimization/choice

Other neural network architectures

Operator learning

Inverse problems

Parametrized PDEs

Geometry-dependant resolution

Hybrid approaches

## References

<a id="ref1"></a>
[1] Maziar Raissi, Paris Perdikaris, George Em Karniadakis (2017). "Physics Informed Deep Learning (Part I): Data-driven Solutions of Nonlinear Partial Differential Equations". [arXiv:1711.10561](https://arxiv.org/abs/1711.10561)

<a id="ref2"></a>
[2] Sifan Wang, Yujun Teng, Paris Perdikaris (2020). "Understanding and mitigating gradient pathologies in physics-informed neural networks". [arXiv:2001.04536](https://arxiv.org/abs/2001.04536)

<a id="ref3"></a>
[3] Lu, Lu, Pengzhan Jin, et George Em Karniadakis (2021). "DeepONet: Learning nonlinear operators for identifying differential equations based on the universal approximation theorem of operators". [Nature Machine Intelligence 3, nᵒ 3 (2021): 218‑29](https://doi.org/10.1038/s42256-021-00302-5)

<a id="ref4"></a>
[4] Cho, Woojin, Minju Jo, Haksoo Lim, *et al* (2024). "Parameterized Physics-informed Neural Networks for Parameterized PDEs". [arXiv:2408.09446](https://doi.org/10.48550/arXiv.2408.09446)

<a id="ref5"></a>
[5] Toscano, Juan Diego, Vivek Oommen, Alan John Varghese, *et al* (2025). "From PINNs to PIKANs: Recent Advances in Physics-Informed Machine Learning". [Machine Learning for Computational Science and Engineering 1, nᵒ 1 (2025): 15](https://doi.org/10.1007/s44379-025-00015-1)

<a id="ref6"></a>
[6] Lu, Lu and Meng, Xuhui and Mao, Zhiping and Karniadakis, George Em (2021). "DeepXDE: A deep learning library for solving differential equations". [SIAM Review 63, nᵒ 1 (2021): 208-228](https://doi.org/10.1137/19M1274067)

<a id="ref7"></a>
[7] Jonathon Hare (accessed in 2026). "Differentiate Automatically: An Introduction to Automatic Differentiation". [University of Southampton handouts, COMP6248](https://comp6248.ecs.soton.ac.uk/handouts/autograd-handouts.pdf)